<a href="https://colab.research.google.com/github/okechukwuchude/Fraud-detection-using-PySpark/blob/main/Copy_of_testing_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
pip install findspark

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=0076ef9b0acc6557e453f50831fab77aaa8e53ee0f7db3a906815b0d500f0fd1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pyspark.sql.functions as F

from pyspark.ml.feature import VectorSlicer, VectorAssembler

In [ ]:
#sc = SparkContext(appName="finicial fraud detection")
#sqlc = SQLContext(sc)

In [ ]:
spark = SparkSession.builder \
    .appName("FinancialFraudDetection") \
    .config("spark.kryoserializer.buffer.max", "512m") \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Synthetic Financial Datasets.csv',header = True)


In [ ]:
data.printSchema()

root
 |-- step: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: string (nullable = true)
 |-- newbalanceOrig: string (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: string (nullable = true)
 |-- newbalanceDest: string (nullable = true)
 |-- isFraud: string (nullable = true)
 |-- isFlaggedFraud: string (nullable = true)



In [ ]:
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Synthetic Financial Datasets.csv',inferSchema=True,header = True)


In [ ]:
data.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [ ]:
data.dtypes

[('step', 'int'),
 ('type', 'string'),
 ('amount', 'double'),
 ('nameOrig', 'string'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('nameDest', 'string'),
 ('oldbalanceDest', 'double'),
 ('newbalanceDest', 'double'),
 ('isFraud', 'int'),
 ('isFlaggedFraud', 'int')]

In [ ]:
data.count()

6362620

In [ ]:
data.groupBy('isFraud').count().show()


+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [ ]:
data.show(5,0)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount  |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|1   |PAYMENT |9839.64 |C1231006815|170136.0     |160296.36     |M1979787155|0.0           |0.0           |0      |0             |
|1   |PAYMENT |1864.28 |C1666544295|21249.0      |19384.72      |M2044282225|0.0           |0.0           |0      |0             |
|1   |TRANSFER|181.0   |C1305486145|181.0        |0.0           |C553264065 |0.0           |0.0           |1      |0             |
|1   |CASH_OUT|181.0   |C840083671 |181.0        |0.0           |C38997010  |21182.0       |0.0           |1      |0             |
|1   |PAYMENT |11668.14|C2048537720|41554.0      |29885.86      |M1230701703|0.0   

In [ ]:
##show fraud vs non fraud
data.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [ ]:
data = data.select("step","type", "amount", "oldbalanceOrg", "newbalanceOrig","oldbalanceDest","newbalanceDest", "isFraud")

In [ ]:
### check if there is missing values in the data
from pyspark.sql.functions import col,sum
data.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in data.columns)).show()

+----+----+------+-------------+--------------+--------------+--------------+-------+
|step|type|amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|
+----+----+------+-------------+--------------+--------------+--------------+-------+
|   0|   0|     0|            0|             0|             0|             0|      0|
+----+----+------+-------------+--------------+--------------+--------------+-------+



In [ ]:
frdtype = data.filter(data['isFraud'] == 0).groupby('type').count().show()


+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 528812|
| CASH_IN|1399284|
|CASH_OUT|2233384|
| PAYMENT|2151495|
|   DEBIT|  41432|
+--------+-------+



In [ ]:
## show which type of transaction is fraudulent
frdtype = data.filter(data['isFraud'] == 1).groupby('type').count().show()


+--------+-----+
|    type|count|
+--------+-----+
|TRANSFER| 4097|
|CASH_OUT| 4116|
+--------+-----+



In [ ]:
##show flagged vs non flagged
#data.groupby('isFlaggedFraud').count().show()

Handling Class Imbalance

In [ ]:
##getting the ratio
major_df = data.filter(col("isFraud") == 0)
minor_df = data.filter(col("isFraud") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 773


there are 773 non-fraudulent transactions for every fraudulent transaction

In [ ]:
from pyspark.sql.functions import explode, array, lit

OverSampling

In [ ]:
a = range(ratio)
# duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')
# combine both oversampled minority rows and previous majority rows
os_combined_df = major_df.unionAll(oversampled_df)

In [ ]:
##show fraud vs non fraud
os_combined_df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|6348649|
|      0|6354407|
+-------+-------+



UnderSampling

In [ ]:
sampled_majority_df = major_df.sample(False, 1/ratio)
us_combined_df = sampled_majority_df.unionAll(minor_df)

In [ ]:
##show fraud vs non fraud
us_combined_df.groupBy('isFraud').count().show()

+-------+-----+
|isFraud|count|
+-------+-----+
|      1| 8213|
|      0| 8282|
+-------+-----+



Convert category to numeric

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler

In [ ]:
index_data=StringIndexer(inputCols=["type"],
                                outputCols=["type_index"])
data_trans=index_data.fit(data).transform(data)
data_trans.show(5)

+----+--------+--------+-------------+--------------+--------------+--------------+-------+----------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|type_index|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+----------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|       1.0|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|       1.0|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|       3.0|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|       0.0|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|       1.0|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+----------+
only showing top 5 rows



In [ ]:
data_Select=data_trans.select(["step", "amount", "oldbalanceOrg", "newbalanceOrig","oldbalanceDest","newbalanceDest",
                               "isFraud", "type_index"])
data_Select.show(5)

+----+-------+-------------+--------------+--------------+--------------+-------+----------+
|step| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|type_index|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+
|   1|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|       1.0|
|   1|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|       1.0|
|   1|  181.0|        181.0|           0.0|           0.0|           0.0|      1|       3.0|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+
only showing top 3 rows



seperating target variable

In [ ]:
feat_col=["step", "amount", "oldbalanceOrg", "newbalanceOrig","oldbalanceDest","newbalanceDest", "type_index"]
fraud_Asmbl=VectorAssembler(inputCols=feat_col, outputCol="out_Fraud")
fraud_Add = fraud_Asmbl.transform(data_Select)
fraud_Add.show(3)

+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+
|step| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|type_index|           out_Fraud|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+
|   1|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|       1.0|[1.0,9839.64,1701...|
|   1|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|       1.0|[1.0,1864.28,2124...|
|   1|  181.0|        181.0|           0.0|           0.0|           0.0|      1|       3.0|[1.0,181.0,181.0,...|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+
only showing top 3 rows



Scaling the data

In [ ]:
fraud_Scaled = StandardScaler(inputCol="out_Fraud", outputCol="scaled_fraud")
fraud_add2= fraud_Scaled.fit(fraud_Add).transform(fraud_Add)
fraud_add2.show(3)

+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+--------------------+
|step| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|type_index|           out_Fraud|        scaled_fraud|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+--------------------+
|   1|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|       1.0|[1.0,9839.64,1701...|[0.00702582836891...|
|   1|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|       1.0|[1.0,1864.28,2124...|[0.00702582836891...|
|   1|  181.0|        181.0|           0.0|           0.0|           0.0|      1|       3.0|[1.0,181.0,181.0,...|[0.00702582836891...|
+----+-------+-------------+--------------+--------------+--------------+-------+----------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
fraud_det_select=fraud_add2.select("scaled_fraud", 'isFraud')
fraud_det_select.show(5)
fraud_det_select.describe().show(3, truncate=False)

+--------------------+-------+
|        scaled_fraud|isFraud|
+--------------------+-------+
|[0.00702582836891...|      0|
|[0.00702582836891...|      0|
|[0.00702582836891...|      1|
+--------------------+-------+
only showing top 3 rows

+-------+--------------------+
|summary|isFraud             |
+-------+--------------------+
|count  |6362620             |
|mean   |0.001290820448180152|
|stddev |0.035904796801604424|
+-------+--------------------+
only showing top 3 rows



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
# model evaluation metics
eval_1 = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="accuracy",)
eval_2 = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="isFraud", metricName="areaUnderROC")
eval_3 = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="f1")
eval_4 = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="weightedPrecision")

Splitting data

In [ ]:
train_data, test_data = fraud_det_select.randomSplit([.7,.3])

Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="isFraud", featuresCol="scaled_fraud",predictionCol='prediction',numTrees= 15 ,maxDepth=15,)
rf_fraud = rf.fit(train_data)

In [ ]:
rf_pred = rf_fraud.transform(test_data)
rf_pred.show(5)

+--------------------+-------+--------------------+--------------------+----------+
|        scaled_fraud|isFraud|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(7,[0,1],[0.07025...|      0|[14.9721148311699...|[0.99814098874466...|       0.0|
|(7,[0,1],[0.09133...|      0|[14.9681921081641...|[0.99787947387760...|       0.0|
|(7,[0,1],[0.12646...|      0|[14.9833028633118...|[0.99888685755412...|       0.0|
|(7,[0,1],[0.12646...|      0|[14.9833028633118...|[0.99888685755412...|       0.0|
|(7,[0,1],[0.28103...|      0|[14.9895881807116...|[0.99930587871411...|       0.0|
+--------------------+-------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
rf_acc1 = (eval_1.evaluate(rf_pred)*100)
print("Test Accuracy: ", rf_acc1 ,'%')
print("Test Error: ", (100-rf_acc1), '%')

rf_acc2 = (eval_2.evaluate(rf_pred)*100)
print("AOC:", rf_acc2,'%')

rf_acc3 = (eval_3.evaluate(rf_pred)*100)
print("f1:", rf_acc3,'%')

rf_acc4 = (eval_4.evaluate(rf_pred)*100)
print("weightedPrecision:", rf_acc4,'%')

Test Accuracy:  99.96615208403085 %
Test Error:  0.03384791596914738 %
AOC: 99.80996709812983 %
f1: 99.96403506136528 %
weightedPrecision: 99.96536577364984 %


In [ ]:
cm_rf = rf_pred.groupBy("isFraud").pivot("prediction", [0.0, 1.0]).count().na.fill(0)
print('Random Forest Confusion Matrix')
cm_rf.show()

Random Forest Confusion Matrix
+-------+-------+----+
|isFraud|    0.0| 1.0|
+-------+-------+----+
|      1|    593|1833|
|      0|1906058|  53|
+-------+-------+----+



Support Vector Machines

In [ ]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(labelCol='isFraud',featuresCol='scaled_fraud',predictionCol='prediction', maxIter=100)
svm_fraud = svm.fit(train_data)
svm_pred = svm_fraud.transform(test_data)
svm_pred.show(5, True)

+--------------------+-------+--------------------+----------+
|        scaled_fraud|isFraud|       rawPrediction|prediction|
+--------------------+-------+--------------------+----------+
|(7,[0,1],[0.07025...|      0|[1.12330692811830...|       0.0|
|(7,[0,1],[0.09133...|      0|[1.13632586637716...|       0.0|
|(7,[0,1],[0.12646...|      0|[1.12758114641391...|       0.0|
|(7,[0,1],[0.12646...|      0|[1.12479092123655...|       0.0|
|(7,[0,1],[0.28103...|      0|[1.12380829189253...|       0.0|
+--------------------+-------+--------------------+----------+
only showing top 5 rows



In [ ]:
svm_acc1 = (eval_1.evaluate(svm_pred)*100)
print("Test accuracy:", svm_acc1,'%')
print("Test Error: ", (100-svm_acc1), '%')

svm_acc2 = (eval_2.evaluate(svm_pred)*100)
print("AOC:", svm_acc2,'%')

svm_acc3 = (eval_3.evaluate(svm_pred)*100)
print("f1:", svm_acc3,'%')

svm_acc4 = (eval_4.evaluate(svm_pred)*100)
print("weightedPrecision:", svm_acc4,'%')

Test accuracy: 99.87419683244286 %
Test Error:  0.12580316755713739 %
AOC: 82.42617090996114 %
f1: 99.81261802222264 %
weightedPrecision: 99.87435509888567 %


In [ ]:
cm_svm = svm_pred.groupBy("isFraud").pivot("prediction", [0.0, 1.0]).count().na.fill(0)
print('svm Confusion Matrix')
cm_svm.show()

svm Confusion Matrix
+-------+-------+---+
|isFraud|    0.0|1.0|
+-------+-------+---+
|      1|   2401| 25|
|      0|1906111|  0|
+-------+-------+---+



Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(labelCol='isFraud',featuresCol='scaled_fraud',predictionCol='prediction')
nb_fraud = nb.fit(train_data)

nb_pred = nb_fraud.transform(test_data)
nb_pred.show(5)

+--------------------+-------+--------------------+--------------------+----------+
|        scaled_fraud|isFraud|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(7,[0,1],[0.07025...|      0|[-1.0779246838839...|[0.99771110697612...|       0.0|
|(7,[0,1],[0.09133...|      0|[-0.1225151734792...|[0.99869170724959...|       0.0|
|(7,[0,1],[0.12646...|      0|[-0.7166052228952...|[0.99820742026334...|       0.0|
|(7,[0,1],[0.12646...|      0|[-0.9170016272766...|[0.99799038793911...|       0.0|
|(7,[0,1],[0.28103...|      0|[-0.8381361356894...|[0.99827124370135...|       0.0|
+--------------------+-------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
nb_acc1 = (eval_1.evaluate(nb_pred)*100)
print("Test accuracy:" , nb_acc1, '%')
print("Test Error: ", (100-nb_acc1), '%')

nb_acc2 = (eval_2.evaluate(nb_pred)*100)
print("AOC", nb_acc2,'%')

nb_acc3 = (eval_3.evaluate(nb_pred)*100)
print("f1:", nb_acc3,'%')

nb_acc4 = (eval_4.evaluate(nb_pred)*100)
print("weightedPrecision:", nb_acc4,'%')

Test accuracy: 99.6891860100171 %
Test Error:  0.3108139899829041 %
AOC 30.904380823023775 %
f1: 99.73044645910518 %
weightedPrecision: 99.77373428197801 %


In [ ]:
cm_nb = nb_pred.groupBy("isFraud").pivot("prediction", [0.0, 1.0]).count().na.fill(0)
print('niave bayes Confusion Matrix')
cm_nb.show()

niave bayes Confusion Matrix
+-------+-------+----+
|isFraud|    0.0| 1.0|
+-------+-------+----+
|      1|   2087| 339|
|      0|1902266|3845|
+-------+-------+----+



Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='isFraud',featuresCol='scaled_fraud',predictionCol='prediction')
lr_fraud = lr.fit(train_data)
lr_pred = lr_fraud.transform(test_data)
lr_pred.show(5)

+--------------------+-------+--------------------+--------------------+----------+
|        scaled_fraud|isFraud|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(7,[0,1],[0.07025...|      0|[10.6390455320028...|[0.99997603867518...|       0.0|
|(7,[0,1],[0.09133...|      0|[8.95540332059600...|[0.99987097860960...|       0.0|
|(7,[0,1],[0.12646...|      0|[9.88741569403132...|[0.99994919249625...|       0.0|
|(7,[0,1],[0.12646...|      0|[10.2300267879392...|[0.99996393049758...|       0.0|
|(7,[0,1],[0.28103...|      0|[9.72698477534100...|[0.99994035167113...|       0.0|
+--------------------+-------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
lr_acc1 = (eval_1.evaluate(lr_pred)*100)
print("Test accuracy:", lr_acc1,'%')
print("Test Error: ", (100-lr_acc1), '%')

lr_acc2 = (eval_2.evaluate(lr_pred)*100)
print("AOC:", lr_acc2,'%')

lr_acc3 = (eval_3.evaluate(lr_pred)*100)
print("f1:", lr_acc3,'%')

lr_acc4 = (eval_4.evaluate(lr_pred)*100)
print("weightedPrecision:", lr_acc4,'%')

Test accuracy: 99.9260166294916 %
Test Error:  0.07398337050840098 %
AOC: 96.85218142505514 %
f1: 99.91556135531849 %
weightedPrecision: 99.91898310691276 %


In [ ]:
cm_lr = lr_pred.groupBy("isFraud").pivot("prediction", [0.0, 1.0]).count().na.fill(0)
print('logistic regression Confusion Matrix')
cm_lr.show()

logistic regression Confusion Matrix
+-------+-------+----+
|isFraud|    0.0| 1.0|
+-------+-------+----+
|      1|   1241|1185|
|      0|1905940| 171|
+-------+-------+----+

